# Домашнее задание к уроку №3
Выполняем парсинг по сайту hh.ru выбор вакансий по запросу 'python', с возможностью отбора по уровню з/п.


In [5]:
import requests
# from lxml import html
from pprint import pprint
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd
import json

In [9]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0'
}
url = 'https://krasnodar.hh.ru/search/vacancy'
params = {
    'text':'python',
    'from':'suggest_post',
    'salary':'',
    'area':'53',
    'page':'0',
    'ored_clusters':'true',
    'enable_snippets':'true'
}

# response = requests.get(url, headers=headers, params=params)


In [10]:
def search_position_on_wage(limit=1, url=url, headers=headers, search_params=params.copy()):
    count = 1
    result_dict = dict()
    # Функция для преобразования поля "Зарплата" в список чисел.
    def string_to_digit(obj):
        result = []
        for el in obj.split(sep='–'):
            spam = []
            for i in list(el):
                if i.isdigit():
                    spam.append(i)
            result.append(int(''.join(spam))) if len(spam) != 0 else result.append(0)
        return result
    
    def search_result_printing(obj: 'dictionary'):
        for k, v in obj.items():
            print(f'{k}. {v[0]} - {v[2]}\n{v[1]}')
    
    while search_params['page'] != '-1':
        answer = requests.get(url, headers=headers, params=search_params)
        dom = bs(answer.content, 'html.parser')
        class_list = dom.find_all('div', {'class':'vacancy-serp-item-body__main-info'})
    
        for item in class_list:
            search_position = item.find('a', {'class':'serp-item__title'}).string
            search_link = item.find('a')['href']
            spam = item.find('span', {'class':'bloko-header-section-3'})
            wage = spam.text if spam else 'Зарплата не указана.'
            foo = string_to_digit(wage)
            record_flag = False
            for j in range(len(foo)):
                if foo[j] >= limit:
                    record_flag = True
            if record_flag:
                result_dict[count] = [search_position, search_link, wage]
                count += 1
        if len(class_list) != 0:
            ham = int(search_params['page']) + 1
            search_params['page'] = str(ham)
        else:
            search_params['page'] = '-1'
        sleep(1)
    search_result_printing(result_dict)

In [11]:
search_position_on_wage(limit=10000)

1. Python разработчик - до 20 000 руб.
https://krasnodar.hh.ru/vacancy/74186083?from=vacancy_search_list&query=python
2. Python developer / Django разработчик - до 140 000 руб.
https://krasnodar.hh.ru/vacancy/72001287?from=vacancy_search_list&query=python
3. Middle Python разработчик (Django) - 160 000 – 240 000 руб.
https://krasnodar.hh.ru/vacancy/67115536?from=vacancy_search_list&query=python
4. Junior Python разработчик (Django) в офис в Краснодаре - от 60 000 руб.
https://krasnodar.hh.ru/vacancy/69467314?from=vacancy_search_list&query=python
5. Senior Python разработчик (Django) - от 280 000 руб.
https://krasnodar.hh.ru/vacancy/69529590?from=vacancy_search_list&query=python
6. Middle DevOps инженер - от 150 000 руб.
https://krasnodar.hh.ru/vacancy/73537066?from=vacancy_search_list&query=python
7. Инженер-программист - 100 000 – 130 000 руб.
https://krasnodar.hh.ru/vacancy/73064117?from=vacancy_search_list&query=python
8. Программист отдела поддержки информационных систем - от 120 0